In [1]:
# Question 1

import gzip
import random
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [2]:
# split the data into training set and validation set

train_set = set()
validation_set = set()
count = 0

for l in readGz("train.json.gz"):    
    count += 1
    if count <= 100000:
        train_set.add(l['reviewerID'] + '-' + l['itemID'])
    else:
        validation_set.add(l['reviewerID'] + '-' + l['itemID'] )


In [3]:
len(train_set)

100000

In [3]:
# simulate validation set

user_sample = []
item_sample = []

for l in readGz("train.json.gz"):
    user_sample.append(l['reviewerID'])
    item_sample.append(l['itemID'])

random.shuffle(user_sample)
random.shuffle(item_sample)

In [4]:
random_user_sample = set()
random_item_sample = set()

for i in range(200000):
    random_user_sample.add(user_sample[i])
    random_item_sample.add(item_sample[i])
    
random_sample = list()
random_user_sample = list(random_user_sample)
random_item_sample = list(random_item_sample)

count = 0

while count < 200000:
    x = random.choice(random_user_sample)
    y = random.choice(random_item_sample)
    random_sample.append(x + '-' + y)
    count += 1

In [5]:
random_sample_set = set()
for i in random_sample:
    random_sample_set.add(i)

add_validation = random_sample_set - validation_set
final_validation = validation_set.copy()
count = 100000

for pairs in add_validation:
    count += 1
    final_validation.add(pairs)
    if count == 200000:
        break
len(final_validation)

200000

In [5]:
# evaluate the performance of the baseline model on the new validation set

businessCount = defaultdict(int)
totalPurchases = 0

for l in train_set:
    user,business = l.strip().split('-')
    businessCount[business] += 1
    totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalPurchases/2: 
    break

label = []
prediction = []
for pairs in final_validation:
    if pairs in train_set or pairs in validation_set:
        label.append(1)
    else:
        label.append(0)

for pairs in final_validation:
    user,item = pairs.strip().split('-')
    if item in return1:
        prediction.append(1)
    else:
        prediction.append(0)

3714

In [7]:
# calculation accuracy

count = 0

for i in range(200000):
    if label[i] == prediction[i]:
        count += 1

accuracy = count/200000
print(accuracy)

0.63029


In [8]:
# Question 2

businessCount = defaultdict(int)
totalPurchases = 0

for l in train_set:
    user,business = l.strip().split('-')
    businessCount[business] += 1
    totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalPurchases*0.52: 
    break

label = []
prediction = []
for pairs in final_validation:
    if pairs in train_set or pairs in validation_set:
        label.append(1)
    else:
        label.append(0)

for pairs in final_validation:
    user,item = pairs.strip().split('-')
    if item in return1:
        prediction.append(1)
    else:
        prediction.append(0)

# calculation accuracy

count = 0

for i in range(200000):
    if label[i] == prediction[i]:
        count += 1

accuracy = count/200000
print("When the threshold becomes 52%, the accuracy will increases to "+str(accuracy))

When the threshold becomes 52%, the accuracy will increases to 0.63073


In [6]:
# Question 3

# map items to categories
user = []
item = []
category = []

for l in readGz("train.json.gz"):
    user.append(l['reviewerID'])
    item.append(l['itemID'])
    category.append(l['categories'])

In [7]:
IC_dict = dict(zip(item,category))

In [8]:
# construct a user-item dictionary from the train set
UI_dict = defaultdict(list)

for l in list(train_set):
    user = l.split('-')[0]
    item = l.split('-')[1]
    UI_dict[user].append(item)

36427

In [12]:
return_item = set()

for user in UI_dict:
    count = 0
    for item in UI_dict[user]:
        count += 1
    for i in range(count):
        for j in range(i+1,count):
            bool_count = any(x in IC_dict[UI_dict[user][i]] for x in IC_dict[UI_dict[user][j]])
            if bool_count == False:
                continue
            else:
                return_item.add(UI_dict[user][i])
                return_item.add(UI_dict[user][j])               

In [13]:
label = []
prediction = []
for pairs in final_validation:
    if pairs in train_set or pairs in validation_set:
        label.append(1)
    else:
        label.append(0)

for pairs in final_validation:
    user,item = pairs.strip().split('-')
    if item in return_item:
        prediction.append(1)
    else:
        prediction.append(0)

In [14]:
# calculation accuracy

count = 0

for i in range(200000):
    if label[i] == prediction[i]:
        count += 1

accuracy = count/200000
print(accuracy)

0.550035


In [15]:
# Question 4

purchase = open("pairs_Purchase.txt","r").readlines()
purchase = purchase[1:]
purchase = [i.strip() for i in purchase]

prediction = []

for pairs in purchase:
    user,item = pairs.strip().split('-')
    if item in return_item:
        prediction.append(1)
    else:
        prediction.append(0)

print(prediction)

[1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 